In [1]:
!pip -q install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import cohere
co = cohere.Client("10I2441fRlW11hAM4E6AmYsKWgZAx4cSqzRfjqsY")

# 1. Diseases

In [6]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [3]:
prompt = """The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the diseases mentioned.

Progressive neurodegeneration of the optic nerve and the loss of retinal ganglion cells is a hallmark of glaucoma, the leading cause of irreversible blindness worldwide, with primary open-angle glaucoma (POAG) being the most frequent form of glaucoma in the Western world. While some genetic mutations have been identified for some glaucomas, those associated with POAG are limited and for most POAG patients, the etiology is still unclear. Unfortunately, treatment of this neurodegenerative disease and other retinal degenerative diseases is lacking. For POAG, most of the treatments focus on reducing aqueous humor formation, enhancing uveoscleral or conventional outflow, or lowering intraocular pressure through surgical means. These efforts, in some cases, do not always lead to a prevention of vision loss and therefore other strategies are needed to reduce or reverse the progressive neurodegeneration. In this review, we will highlight some of the ocular pharmacological approaches that are being tested to reduce neurodegeneration and provide some form of neuroprotection.

List of extracted diseases:"""

#### Base (Cohere)

In [4]:
response = co.generate(
    model='command-nightly',
    prompt = prompt,
    max_tokens=200,
    temperature=0.750)

base_model = response.generations[0].text

print( base_model )

 - Glaucoma
- Primary open-angle glaucoma
- Retinal degenerative diseases


#### Custom

In [5]:
response = co.generate(
    model='1715f1cb-3294-46a2-8631-5182031e6a5d-ft'
    prompt = prompt,
    max_tokens=200,
    temperature=0.750)

diesease_model = response.generations[0].text

print( diesease_model )

SyntaxError: ignored

# 2. Chemicals

In [ ]:
prompt = """The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the chemicals mentioned.

To test the validity of the hypothesis that hypomethylation of DNA plays an important role in the initiation of carcinogenic process, 5-azacytidine (5-AzC) (10 mg/kg), an inhibitor of DNA methylation, was given to rats during the phase of repair synthesis induced by the three carcinogens, benzo[a]-pyrene (200 mg/kg), N-methyl-N-nitrosourea (60 mg/kg) and 1,2-dimethylhydrazine (1,2-DMH) (100 mg/kg). The initiated hepatocytes in the liver were assayed as the gamma-glutamyltransferase (gamma-GT) positive foci formed following a 2-week selection regimen consisting of dietary 0.02% 2-acetylaminofluorene coupled with a necrogenic dose of CCl4. The results obtained indicate that with all three carcinogens, administration of 5-AzC during repair synthesis increased the incidence of initiated hepatocytes, for example 10-20 foci/cm2 in 5-AzC and carcinogen-treated rats compared with 3-5 foci/cm2 in rats treated with carcinogen only. Administration of [3H]-5-azadeoxycytidine during the repair synthesis induced by 1,2-DMH further showed that 0.019 mol % of cytosine residues in DNA were substituted by the analogue, indicating that incorporation of 5-AzC occurs during repair synthesis. In the absence of the carcinogen, 5-AzC given after a two thirds partial hepatectomy, when its incorporation should be maximum, failed to induce any gamma-GT positive foci. The results suggest that hypomethylation of DNA per se may not be sufficient for initiation. Perhaps two events might be necessary for initiation, the first caused by the carcinogen and a second involving hypomethylation of DNA.

List of extracted chemicals:"""

#### Base (Cohere)

In [ ]:
response = co.generate(
    model='command-nightly',
    prompt = prompt,
    max_tokens=200,
    temperature=0.750)

base_model = response.generations[0].text

print( base_model )

 - 5-azacytidine (5-AzC)
- benzo[a]-pyrene
- N-methyl-N-nitrosourea
- 1,2-dimethylhydrazine
- CCl4
- 2-acetylaminofluorene


#### Custom

In [8]:
import json

# Open the JSON file
with open('/bin/bc5cdr.json') as json_file:
    # Load the JSON data
    data = json.load(json_file)

# Print the first data point
print(data[0])


{'passages': [{'document_id': '227508', 'type': 'title', 'text': 'Naloxone reverses the antihypertensive effect of clonidine.', 'entities': [{'id': '0', 'offsets': [[0, 8]], 'text': ['Naloxone'], 'type': 'Chemical', 'normalized': [{'db_name': 'MESH', 'db_id': 'D009270'}]}, {'id': '1', 'offsets': [[49, 58]], 'text': ['clonidine'], 'type': 'Chemical', 'normalized': [{'db_name': 'MESH', 'db_id': 'D003000'}]}], 'relations': [{'id': 'R0', 'type': 'CID', 'arg1_id': 'D008750', 'arg2_id': 'D007022'}]}, {'document_id': '227508', 'type': 'abstract', 'text': 'In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) 

In [9]:
instruction = "The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the diseases mentioned.\n\n"
output_instruction = "\n\nList of extracted diseases:\n"

In [12]:
the_list = []

for item in data:
    if item['dataset_type'] != "test":
        continue  # Don't use test set

    dis = []  # Initialize an empty list for diseases

    # Extract the disease names
    for ent in item['passages'][1]['entities']:  # Iterate through annotations
        if ent['type'] == "Disease":  # Filter for disease entities
            if ent['text'][0] not in dis:  # Remove duplicate diseases in a text
                dis.append(ent['text'][0])  # Add unique disease to the list

    the_list.append(  # Create a dictionary and append it to the list
        {'prompt': instruction +
         item['passages'][1]['text'] +
         output_instruction,
         'completion': "- " + "\n- ".join(dis)}  # Format completion with "- "
    )


In [13]:
# Writing to sample.json
with open("disease_instruct_all.jsonl", "w") as outfile:
  for item in the_list:
    outfile.write(json.dumps(item) + "\n")

In [14]:
from google.colab import files
files.download('disease_instruct_all.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
response = co.generate(
    model='1715f1cb-3294-46a2-8631-5182031e6a5d-ft',
    prompt = prompt,
    max_tokens=200,
    temperature=0.750)

diesease_model = response.generations[0].text

print( diesease_model )

 - 5-azacytidine
- 5-AzC
- benzo[a]-pyrene
- N-methyl-N-nitrosourea
- 1,2-dimethylhydrazine
- 1,2-DMH
- 2-acetylaminofluorene
- CCl4
- [3H]-5-azadeoxycytidine
- cytosine


# 3. Relationships

In [ ]:
prompt = """The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the influences between the chemicals and diseases mentioned.

The yield of severe cirrhosis of the liver (defined as a shrunken finely nodular liver with micronodular histology, ascites greater than 30 ml, plasma albumin less than 2.2 g/dl, splenomegaly 2-3 times normal, and testicular atrophy approximately half normal weight) after 12 doses of carbon tetrachloride given intragastrically in the phenobarbitone-primed rat was increased from 25% to 56% by giving the initial "calibrating" dose of carbon tetrachloride at the peak of the phenobarbitone-induced enlargement of the liver. At this point it was assumed that the cytochrome P450/CCl4 toxic state was both maximal and stable. The optimal rat size to begin phenobarbitone was determined as 100 g, and this size as a group had a mean maximum relative liver weight increase 47% greater than normal rats of the same body weight. The optimal time for the initial dose of carbon tetrachloride was after 14 days on phenobarbitone.

nList of extracted influences:"""

#### Base (Cohere)

In [ ]:
response = co.generate(
    model='command-nightly',
    prompt = prompt,
    max_tokens=200,
    temperature=0.750)

base_model = response.generations[0].text

print( base_model )

 severe cirrhosis of the liver influences shrinking, finely nodular, ascites, plasma albumin, splenomegaly, testicular atrophy, carbon tetrachloride, phenobarbitone


#### Custom

In [ ]:
response = co.generate(
    model='1715f1cb-3294-46a2-8631-5182031e6a5d-ft',
    prompt = prompt,
    max_tokens=200,
    temperature=0.750)

diesease_model = response.generations[0].text

print( diesease_model )

 - Chemical phenobarbitone influences disease cirrhosis of the liver
- Chemical carbon tetrachloride influences disease cirrhosis of the liver
